In [6]:
import numpy as np
import cv2 as cv
from glob import glob

In [9]:
img_mask = './images/left??.jpg'

In [11]:
img_names = glob(img_mask)

In [13]:
square_size = 1.0

pattern_size = (9, 6)
pattern_points = np.zeros((np.prod(pattern_size), 3), np.float32)
pattern_points[:, :2] = np.indices(pattern_size).T.reshape(-1, 2)
pattern_points *= square_size

obj_points = []
img_points = []
h, w = cv.imread(img_names[0], 0).shape[:2]

In [16]:
def processImage(fn):
    print('processing %s... ' % fn)
    img = cv.imread(fn, 0)
    if img is None:
        print("Failed to load", fn)
        return None

    assert w == img.shape[1] and h == img.shape[0], ("size: %d x %d ... " % (img.shape[1], img.shape[0]))
    found, corners = cv.findChessboardCorners(img, pattern_size)
    if found:
        term = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_COUNT, 30, 0.1)
        cv.cornerSubPix(img, corners, (5, 5), (-1, -1), term)

#     if debug_dir:
#         vis = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
#         cv.drawChessboardCorners(vis, pattern_size, corners, found)
#         _path, name, _ext = splitfn(fn)
#         outfile = os.path.join(debug_dir, name + '_chess.png')
#         cv.imwrite(outfile, vis)

    if not found:
        print('chessboard not found')
        return None

    print('           %s... OK' % fn)
    return (corners.reshape(-1, 2), pattern_points)

In [19]:
chessboards = [processImage(fn) for fn in img_names]

processing ./images/left09.jpg... 
           ./images/left09.jpg... OK
processing ./images/left08.jpg... 
           ./images/left08.jpg... OK
processing ./images/left14.jpg... 
           ./images/left14.jpg... OK
processing ./images/left01.jpg... 
           ./images/left01.jpg... OK
processing ./images/left03.jpg... 
           ./images/left03.jpg... OK
processing ./images/left02.jpg... 
           ./images/left02.jpg... OK
processing ./images/left12.jpg... 
           ./images/left12.jpg... OK
processing ./images/left06.jpg... 
           ./images/left06.jpg... OK
processing ./images/left07.jpg... 
           ./images/left07.jpg... OK
processing ./images/left13.jpg... 
           ./images/left13.jpg... OK
processing ./images/left05.jpg... 
           ./images/left05.jpg... OK
processing ./images/left11.jpg... 
           ./images/left11.jpg... OK
processing ./images/left04.jpg... 
           ./images/left04.jpg... OK


In [20]:
chessboards = [x for x in chessboards if x is not None]
for (corners, pattern_points) in chessboards:
    img_points.append(corners)
    obj_points.append(pattern_points)

In [22]:
# calculate camera distortion
rms, camera_matrix, dist_coefs, rvecs, tvecs = cv.calibrateCamera(obj_points, img_points, (w, h), None, None)

print("\nRMS:", rms)
print("camera matrix:\n", camera_matrix)
print("distortion coefficients: ", dist_coefs.ravel())


RMS: 0.19643790890334045
camera matrix:
 [[532.79536562   0.         342.45825163]
 [  0.         532.91928338 233.90060514]
 [  0.           0.           1.        ]]
distortion coefficients:  [-2.81086258e-01  2.72581010e-02  1.21665908e-03 -1.34204274e-04
  1.58514023e-01]
